In [1]:
import time      
import os 
import requests
import logging
import threading
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import json
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
#Get latitude & longitude from wikipedia for each alarm
#With Crawling
def landmarksInfo(name) :
    r = requests.get("https://he.wikipedia.org/wiki/" + name)
    soup = BeautifulSoup(r.content)
    latitude = soup.find("span", attrs={"class": "latitude"})
    longitude = soup.find("span", attrs={"class": "longitude"})
    print
    if(latitude == None):
        latitude="unidentified"
        longitude="unidentified"
    else:
        latitude=latitude.get_text()
        longitude=longitude.get_text()
        
    return latitude, longitude

In [4]:
#Get IDPLACE & DISTRCIT &  from & FORMS & PEOPLE NUMBERT from Central Bureau of Statistics
#For each alarm 
#With API
def officeStatisticsInfo(name) :
    url = "https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&filters=%7B%22Years%22:2019%7D&filtersearch=" + name + "&language=Hebrew&mode=GridData&pageNumber=1&search=&subject=BaseData"
    r = requests.get(url)
    data = r.json()
    data = data['Table']
    num=-1
    if not data: 
        idPlace="unidentified"
        district="unidentified"
        form="unidentified"
        PepoleNumber="unidentified"
    else:
        for i in range(len(data)):
            curName = [r['Name'] for r in data][i]
            if(curName == name):
                num = i 
        idPlace = [r['SemelYishuv'] for r in data][num]
        district = [r['Machoz'] for r in data][num]
        form = [r['TzuratYishuv'] for r in data][num]
        PepoleNumber = [r['PepoleNumber'] for r in data][num]
        
    return idPlace, district, form, PepoleNumber

In [5]:
#Get disatnce from Gaza from "Tremp" website for each alarm 
#With Crawling
def findDistance(name) :
    r = requests.get("https://www.tremp.co.il/distance/distance.php?from=עזה&to=" + name)
    soup = BeautifulSoup(r.content) 
    title = soup.find("p", attrs={"class": "center_xxlarge_font"})
    if(title == None):
        distance="unidentified"
    else:
        distance =  title.find("span", attrs={"style": "color:Navy"}).get_text()
        
    return distance

In [6]:
#The data was taken with api in the Operation “Guardian of the Walls” and saved in the excel 
#Because the history on the website "National Emergency Portal" is a month back
df_AlarmsBeforeClean = pd.DataFrame(pd.read_excel("df_Alarms.xlsx"))

In [7]:
#Data cleaning - names
for index, row in df_AlarmsBeforeClean.iterrows():
        newName = df_AlarmsBeforeClean["names"][index].split(" - ", 1)[0]
        df_AlarmsBeforeClean["names"][index] = newName
        
for index, row in df_AlarmsBeforeClean.iterrows():
    newName = df_AlarmsBeforeClean["names"][index].split(",", 1)[0]
    df_AlarmsBeforeClean["names"][index] = newName

for index, row in df_AlarmsBeforeClean.iterrows():
    name = df_AlarmsBeforeClean["names"][index]
    if(name != "גבעת וושינגטון" and name!="כפר ורבורג" and name!="עין ורד"):
        newName = df_AlarmsBeforeClean["names"][index].split(" ו", 1)[0]
        df_AlarmsBeforeClean["names"][index] = newName
        
for index, row in df_AlarmsBeforeClean.iterrows():
    name = row['names']
    newName = (name.replace("''", '"'))
    newName = (newName.replace('מועצה אזורית ', ''))
    newName = (newName.replace('אזור תעשייה הדרומי ', ''))
    newName = (newName.replace(' גבעולים מלילות', ''))
    newName = (newName.replace(' עמיעוז ישע', ''))
    newName = (newName.replace('אזור תעשייה צפוני ', ''))
    newName = (newName.replace('אזור תעשייה ', ''))
    newName = (newName.replace('תעשיון ', ''))
    newName = (newName.replace('פארק תעשייה ', ''))
    newName = (newName.replace('פארק תעשיות ', ''))
    newName = (newName.replace('כפר נוער ', ''))
    newName = (newName.replace('מטווח ', ''))
    newName = (newName.replace(' והפזורה', ''))
    newName = (newName.replace(' ופזורה', ''))
    newName = (newName.replace('מעון ', ''))
    newName = (newName.replace('מתחם ', ''))
    newName = (newName.replace('תחנת רכבת ' , ''))
    newName = (newName.replace('פנימיית ', ''))
    newName = (newName.replace(' צפוני', ''))
    if(newName.find("א-") == -1):
        newName = (newName.replace('-', ' '))
    df_AlarmsBeforeClean["names"][index] = newName

#df_AlarmsBeforeClean.to_excel("df_Alarms.xlsx")

In [8]:
df_Alarms = df_AlarmsBeforeClean

In [9]:
#For each alaram call the functions - officeStatisticsInfo, findDistance and landmarksInfo
#Add the information to the correct rows

lenAlarms = len(df_Alarms.index)
latitudes=[None] * lenAlarms
longitudes=[None] * lenAlarms
idPlaces=[None] * lenAlarms
districts=[None] * lenAlarms
forms=[None] * lenAlarms
PeopleNumbers=[None] * lenAlarms
distances=[None] * lenAlarms

df_Alarms['latitudes'] =  latitudes
df_Alarms['longitudes'] =  longitudes
df_Alarms['idPlaces'] =  idPlaces
df_Alarms['districts'] =  districts
df_Alarms['forms'] =  forms
df_Alarms['PeopleNumbers'] =  PeopleNumbers
df_Alarms['distances'] =  distances

for index, row in df_Alarms.iterrows():
    name = row['names']
    if df_Alarms["latitudes"][index] == None:
        latitude,longitude = landmarksInfo(name)
        idPlace, district, form, PeopleNumber= officeStatisticsInfo(name)
        distance = findDistance(name)
        if idPlace=="unidentified":
            if(name.find("-")!= -1):
                newName = (name.replace('-', ' '))
                idPlace, district, form, PeopleNumber= officeStatisticsInfo(newName)
        if idPlace=="unidentified":
            if(name.find('"')!= -1):
                newName = (name.replace('"', ''))
                idPlace, district, form, PeopleNumber= officeStatisticsInfo(newName)
        if idPlace=="unidentified":
            if(name.find('י')!= -1):
                newName = (name.replace('י', 'יי'))
                idPlace, district, form, PeopleNumber= officeStatisticsInfo(newName) 
        if idPlace=="unidentified":
            if(name.find('י')!= -1):
                newName = (name.replace('יי', 'י'))
                idPlace, district, form, PeopleNumber= officeStatisticsInfo(newName) 
        if idPlace=="unidentified":
            if(name.find(' ')!= -1):
                newName = (name.replace(' ', '-'))
                idPlace, district, form, PeopleNumber= officeStatisticsInfo(newName) 
        if name=="יהוד מונסון":
                idPlace, district, form, PeopleNumber= officeStatisticsInfo("יהוד") 
        if name=="שהם":
                idPlace, district, form, PeopleNumber= officeStatisticsInfo("שוהם")
        if name=="גבעת חן":
                idPlace, district, form, PeopleNumber= officeStatisticsInfo('גבעת ח"ן') 
        if name=="קצר-א-סיר":
                idPlace, district, form, PeopleNumber= officeStatisticsInfo('קצר א-סר')
        if name=="ראם":
                idPlace, district, form, PeopleNumber= officeStatisticsInfo('בני ראם')
            
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (קיבוץ)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (מושב)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (יישוב)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (מוסד חינוכי)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (מתחם)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo("גשר " + name)
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo("כפר " + name)
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (עיר)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (מרכז כפרי)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (מושבה)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (יישוב קהילתי)")
        if latitude=="unidentified":
            latitude,longitude = landmarksInfo(name+ " (תל מונד)")
        if latitude=="unidentified":
            if(newName.find("-")!= -1):
                name = (name.replace('-', ' '))
                latitude,longitude = landmarksInfo(newName)
        if latitude=="unidentified":
            if(newName.find('"')!= -1):
                name = (name.replace('"', ''))
                latitude,longitude = landmarksInfo(newName)
        df_Alarms["latitudes"][index] = latitude
        df_Alarms["longitudes"][index] = longitude
        df_Alarms["idPlaces"][index] = idPlace
        df_Alarms["districts"][index] = district
        df_Alarms["forms"][index] = form
        df_Alarms["PeopleNumbers"][index] = PeopleNumber
        df_Alarms["distances"][index] = distance
        
        for index2 in range (index+1, lenAlarms):
            if df_Alarms["names"][index2] == name:
                df_Alarms["latitudes"][index2] = df_Alarms["latitudes"][index]
                df_Alarms["longitudes"][index2] = df_Alarms["longitudes"][index]
                df_Alarms["idPlaces"][index2] = df_Alarms["idPlaces"][index]
                df_Alarms["districts"][index2] = df_Alarms["districts"][index]
                df_Alarms["forms"][index2] = df_Alarms["forms"][index]
                df_Alarms["PeopleNumbers"][index2] = df_Alarms["PeopleNumbers"][index]
                df_Alarms["distances"][index2] = df_Alarms["distances"][index]
        
df_Alarms.head(7000)

,names,times,dates,latitudes,longitudes,idPlaces,districts,forms,PeopleNumbers,distances
0,צוחר,01:49,21.05.2021,31°14′11″N,34°25′33″E,1136,הדרום,יישובים כפריים אחרים יהודיים,444,34.9
1,מבטחים,01:49,21.05.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439,35.2
2,כיסופים,01:49,21.05.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280,22.6
3,סופה,01:34,21.05.2021,31°14′14″N,34°20′32″E,1238,הדרום,קיבוצים,211,38.4
4,פרי גן,01:34,21.05.2021,31°13′18″N,34°21′17″E,1231,הדרום,מושבים - לרבות כפרים שיתופיים,246,39.3
...,...,...,...,...,...,...,...,...,...,...
6995,מפלסים,18:15,10.05.2021,31°30′07″N,34°33′43″E,668,הדרום,קיבוצים,"1,023",4
6996,ניר עם,18:15,10.05.2021,31°31′11″N,34°34′48″E,348,הדרום,קיבוצים,611,3.6
6997,רווחה,18:03,10.05.2021,31°38′58″N,34°44′00″E,2051,הדרום,מושבים - לרבות כפרים שיתופיים,551,21.5
6998,"ניר ח""ן",18:03,10.05.2021,31°36′32″N,34°42′53″E,11,הדרום,מושבים - לרבות כפרים שיתופיים,548,17.8


In [10]:
#Data cleaning - PeopleNumbers
for index, row in df_Alarms.iterrows():
    numPeople = row['PeopleNumbers']
    numPeople = (numPeople.replace(",", ''))
    df_Alarms["PeopleNumbers"][index] = numPeople
    
    if (numPeople=="unidentified" or numPeople=="-"): 
        df_Alarms["PeopleNumbers"][index] = None

df_Alarms.PeopleNumbers = df_Alarms.PeopleNumbers.astype(float)

avgNumPeople = df_Alarms["PeopleNumbers"].mean()
df_Alarms['PeopleNumbers'] = df_Alarms['PeopleNumbers'].fillna(avgNumPeople)
df_Alarms.head(8000)

,names,times,dates,latitudes,longitudes,idPlaces,districts,forms,PeopleNumbers,distances
0,צוחר,01:49,21.05.2021,31°14′11″N,34°25′33″E,1136,הדרום,יישובים כפריים אחרים יהודיים,444.000000,34.9
1,מבטחים,01:49,21.05.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.2
2,כיסופים,01:49,21.05.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.6
3,סופה,01:34,21.05.2021,31°14′14″N,34°20′32″E,1238,הדרום,קיבוצים,211.000000,38.4
4,פרי גן,01:34,21.05.2021,31°13′18″N,34°21′17″E,1231,הדרום,מושבים - לרבות כפרים שיתופיים,246.000000,39.3
...,...,...,...,...,...,...,...,...,...,...
7073,ניר יצחק,02:12,24.04.2021,31°14′16″N,34°21′28″E,402,הדרום,קיבוצים,630.000000,37.9
7074,מבטחים,02:11,24.04.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.2
7075,כפר מימון,01:50,24.04.2021,31°25′58″N,34°32′07″E,1095,הדרום,מושבים - לרבות כפרים שיתופיים,375.000000,11.4
7076,כיסופים,22:59,23.04.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.6


In [11]:
#Data cleaning - distances
for index, row in df_Alarms.iterrows():
    distance = row['distances']
    
    if (distance=="unidentified"): 
        df_Alarms["distances"][index] = None

df_Alarms.distances = df_Alarms.distances.astype(float)

avgDistance = df_Alarms["distances"].mean()
df_Alarms['distances'] = df_Alarms['distances'].fillna(avgDistance)
df_Alarms.head(8000)

,names,times,dates,latitudes,longitudes,idPlaces,districts,forms,PeopleNumbers,distances
0,צוחר,01:49,21.05.2021,31°14′11″N,34°25′33″E,1136,הדרום,יישובים כפריים אחרים יהודיים,444.000000,34.900000
1,מבטחים,01:49,21.05.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.200000
2,כיסופים,01:49,21.05.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.600000
3,סופה,01:34,21.05.2021,31°14′14″N,34°20′32″E,1238,הדרום,קיבוצים,211.000000,38.400000
4,פרי גן,01:34,21.05.2021,31°13′18″N,34°21′17″E,1231,הדרום,מושבים - לרבות כפרים שיתופיים,246.000000,39.300000
...,...,...,...,...,...,...,...,...,...,...
7073,ניר יצחק,02:12,24.04.2021,31°14′16″N,34°21′28″E,402,הדרום,קיבוצים,630.000000,37.900000
7074,מבטחים,02:11,24.04.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.200000
7075,כפר מימון,01:50,24.04.2021,31°25′58″N,34°32′07″E,1095,הדרום,מושבים - לרבות כפרים שיתופיים,375.000000,11.400000
7076,כיסופים,22:59,23.04.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.600000


In [12]:
#Data cleaning - idPlaces
for index, row in df_Alarms.iterrows():
    idPlace = row['idPlaces']
    
    if (idPlace=="unidentified"): 
        df_Alarms["idPlaces"][index] = "0"

df_Alarms.head(8000)

,names,times,dates,latitudes,longitudes,idPlaces,districts,forms,PeopleNumbers,distances
0,צוחר,01:49,21.05.2021,31°14′11″N,34°25′33″E,1136,הדרום,יישובים כפריים אחרים יהודיים,444.000000,34.900000
1,מבטחים,01:49,21.05.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.200000
2,כיסופים,01:49,21.05.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.600000
3,סופה,01:34,21.05.2021,31°14′14″N,34°20′32″E,1238,הדרום,קיבוצים,211.000000,38.400000
4,פרי גן,01:34,21.05.2021,31°13′18″N,34°21′17″E,1231,הדרום,מושבים - לרבות כפרים שיתופיים,246.000000,39.300000
...,...,...,...,...,...,...,...,...,...,...
7073,ניר יצחק,02:12,24.04.2021,31°14′16″N,34°21′28″E,402,הדרום,קיבוצים,630.000000,37.900000
7074,מבטחים,02:11,24.04.2021,31°14′25″N,34°24′29″E,829,הדרום,מושבים - לרבות כפרים שיתופיים,439.000000,35.200000
7075,כפר מימון,01:50,24.04.2021,31°25′58″N,34°32′07″E,1095,הדרום,מושבים - לרבות כפרים שיתופיים,375.000000,11.400000
7076,כיסופים,22:59,23.04.2021,31°22′31″N,34°23′57″E,840,הדרום,קיבוצים,280.000000,22.600000


In [13]:
df_Alarms.to_excel("fullDF.xlsx") 